In [1]:
from nlp_to_sql_utils import LLMsToDataBase
from open_ai_utils import enviar_promt_chat_completions_mode, get_embedding, simular_respuesta_generativa
from calcular_tokens import num_tokens_from_string, num_tokens_from_messages

import warnings

# Ignorar todos los warnings
warnings.filterwarnings('ignore')

import pandas as pd
import openai
import numpy as np
import os

In [2]:
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Acceder a la API key
api_key = os.getenv("API_KEY")
openai.api_key = api_key

## Carga de metadatos de la BBDD

Si el archivo.pickle existe en el directorio con los metadatos nos dice que ya existe, de lo contrario, los extrae de la bbdd. LA BASE DE DATOS DEBE ESTAR LEVANTADA 

In [3]:
pagila = {
    'bbdd_name':'pagila', 
    'user'     :'postgres', 
    'password' :'123', 
    'host'     :'localhost', 
    'port'     :'5432'
}

rna_central = {
    'bbdd_name': 'pfmegrnargs', 
    'user'     : 'reader', 
    'password' : 'NWDMCE5xdipIjRrp', 
    'host'     : 'hh-pgsql-public.ebi.ac.uk', 
    'port'     : '5433'
}


llm_to_pagila = LLMsToDataBase(**pagila)
llm_to_pagila.store_metadata_df_to_pickle()
#llm_to_pagila.close_connection()


llm_to_rna = LLMsToDataBase(**rna_central)
llm_to_rna.store_metadata_df_to_pickle()
#llm_to_rna.close_connection()


El archivo ya existe en: ../data/nlp_to_sql/pagila_simple_metadata.pickle
El archivo ya existe en: ../data/nlp_to_sql/pagila_full_metadata.pickle

Proceso de almacenamiento de metadatos para pagila completado.

El archivo ya existe en: ../data/nlp_to_sql/pfmegrnargs_simple_metadata.pickle
El archivo ya existe en: ../data/nlp_to_sql/pfmegrnargs_full_metadata.pickle

Proceso de almacenamiento de metadatos para pfmegrnargs completado.



In [4]:
data_path = '../data/nlp_to_sql/'

simple_md_df = pd.read_pickle(data_path + 'pagila_simple_metadata.pickle')   # pfmegrnargs   # pagila
full_md_df = pd.read_pickle(data_path + 'pagila_full_metadata.pickle')

display(simple_md_df.head())
display(full_md_df.head())

,metadata_str,md_str_tokens,embedding
film,"{""film"": [""film_id"", ""title"", ""description"", ""...",53,"[-0.010331695899367332, 0.012449457310140133, ..."
language,"{""language"": [""language_id"", ""name"", ""last_upd...",14,"[-0.02983446605503559, 0.018869545310735703, 0..."
actor,"{""actor"": [""actor_id"", ""first_name"", ""last_nam...",19,"[-0.0329330638051033, -0.005584632977843285, 0..."
store,"{""store"": [""store_id"", ""manager_staff_id"", ""ad...",20,"[-0.029191363602876663, 0.008534164167940617, ..."
address,"{""address"": [""address_id"", ""address"", ""address...",32,"[-0.027424100786447525, 0.024846235290169716, ..."


,metadata_str,md_str_tokens,embedding
film,"table_name:film\n{""col_name"": ""film_id"", ""type...",443,"[-0.0006032844539731741, 0.024174265563488007,..."
language,"table_name:language\n{""col_name"": ""language_id...",103,"[0.0009828200563788414, 0.027820346876978874, ..."
actor,"table_name:actor\n{""col_name"": ""actor_id"", ""ty...",138,"[-0.0035037612542510033, 0.0059963702224195, 0..."
store,"table_name:store\n{""col_name"": ""store_id"", ""ty...",139,"[-0.006375625263899565, 0.021333562210202217, ..."
address,"table_name:address\n{""col_name"": ""address_id"",...",271,"[-0.0031455650459975004, 0.02590346895158291, ..."


In [5]:
print('El numero de tablas que tiene la actual base de datos es: ')
print(len(simple_md_df), len(full_md_df))

El numero de tablas que tiene la actual base de datos es: 
15 15


In [6]:
# consulta facil - medio
consulta_pagila = '''
dame los nombres de los actores y actrices que hayan participado en 
peliculas con una duracion superior 180 minutos. incluye tambien la 
duración exacta de la pelicula
'''
# consulta facil
consulta_pagila_01 = '''cuantas peliculas tienen la duración maxima y cuanta es esa duración'''

# consulta dificil media
consulta_pagila_02 = '''Los nombres de los actores que hayan participado en más de 3 peliculas, \
                        cuyo nombre empieze por una vocal, y que las peliculas duren más de 180 minutos'''

#consulta dificil
consulta_pagila_03 = '''¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas, \
                        devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas?'''

# consulta_rna_central_1 = "Muestra todas las secuencias de ARN que tienen una longitud mayor a 200 nucleótidos."
# consulta_rna_central_2 =  "Encuentra las entradas de ARN que han sido identificadas en organismos específicos como 'Homo sapiens'."
# consulta_rna_central_3 = "Lista los estudios que han citado secuencias de ARN particulares más de 50 veces."
# consulta_rna_central_4 =  "Obtén los nombres de los investigadores que han contribuido con más de 10 secuencias de ARN a la base de datos."
# consulta_rna_central_5 = "Muestra las secuencias de ARN que se han descubierto en el último año."

token_limit = 1000
n_tablas = None

most_similar_md_simple = LLMsToDataBase.search_most_similar_metadata(
    prompt= consulta_pagila_03, 
    metadata=simple_md_df, 
    n_resultados=n_tablas, 
    lim_tokens=token_limit)

most_similar_md_full = LLMsToDataBase.search_most_similar_metadata(
    prompt= consulta_pagila_03, 
    metadata=full_md_df, 
    n_resultados=n_tablas, 
    lim_tokens=token_limit )

display(most_similar_md_simple)
display(most_similar_md_full)

,metadata_str,md_str_tokens,similarity,embedding,token_cumsum
film,"{""film"": [""film_id"", ""title"", ""description"", ""...",53,0.722164,"[-0.010331695899367332, 0.012449457310140133, ...",53
film_actor,"{""film_actor"": [""actor_id"", ""film_id"", ""last_u...",16,0.720345,"[-0.03354605287313461, 0.0032554285135120153, ...",69
actor,"{""actor"": [""actor_id"", ""first_name"", ""last_nam...",19,0.701002,"[-0.0329330638051033, -0.005584632977843285, 0...",88
film_category,"{""film_category"": [""film_id"", ""category_id"", ""...",16,0.688365,"[-0.025598706677556038, 0.014962821267545223, ...",104
inventory,"{""inventory"": [""inventory_id"", ""film_id"", ""sto...",19,0.684652,"[-0.017722617834806442, 0.002165640937164426, ...",123
rental,"{""rental"": [""rental_id"", ""rental_date"", ""inven...",34,0.673435,"[-0.02543514594435692, 0.002907260088250041, 0...",157
payment,"{""payment"": [""payment_id"", ""customer_id"", ""sta...",27,0.662887,"[-0.02886904962360859, 0.01586032472550869, 0....",184
language,"{""language"": [""language_id"", ""name"", ""last_upd...",14,0.654629,"[-0.02983446605503559, 0.018869545310735703, 0...",198
staff,"{""staff"": [""staff_id"", ""first_name"", ""last_nam...",42,0.654021,"[-0.01896057464182377, 0.008822894655168056, -...",240
category,"{""category"": [""category_id"", ""name"", ""last_upd...",14,0.650922,"[-0.030227964743971825, 0.017394820228219032, ...",254


,metadata_str,md_str_tokens,similarity,embedding,token_cumsum
film_actor,"table_name:film_actor\n{""col_name"": ""actor_id""...",107,0.732734,"[-0.00541864987462759, 0.010073642246425152, 0...",107
actor,"table_name:actor\n{""col_name"": ""actor_id"", ""ty...",138,0.731905,"[-0.0035037612542510033, 0.0059963702224195, 0...",245
film,"table_name:film\n{""col_name"": ""film_id"", ""type...",443,0.719627,"[-0.0006032844539731741, 0.024174265563488007,...",688
film_category,"table_name:film_category\n{""col_name"": ""film_i...",107,0.706705,"[0.0017659964505583048, 0.02148573473095894, -...",795
inventory,"table_name:inventory\n{""col_name"": ""inventory_...",138,0.697546,"[0.00014501469559036195, 0.015438942238688469,...",933


In [7]:
print(len(most_similar_md_simple))
print(len(most_similar_md_full))

15
5


In [8]:
simple_md_str = LLMsToDataBase.get_string_from_metadata_df(most_similar_md_simple)
full_md_str = LLMsToDataBase.get_string_from_metadata_df(most_similar_md_full)

print('SIMPLE METADATA')
print(simple_md_str)
print('\n')
print('FULL METADATA')
print(full_md_str)

SIMPLE METADATA
{"film": ["film_id", "title", "description", "release_year", "language_id", "rental_duration", "rental_rate", "length", "replacement_cost", "rating", "last_update", "special_features", "fulltext"]}
{"film_actor": ["actor_id", "film_id", "last_update"]}
{"actor": ["actor_id", "first_name", "last_name", "last_update"]}
{"film_category": ["film_id", "category_id", "last_update"]}
{"inventory": ["inventory_id", "film_id", "store_id", "last_update"]}
{"rental": ["rental_id", "rental_date", "inventory_id", "customer_id", "return_date", "staff_id", "last_update"]}
{"payment": ["payment_id", "customer_id", "staff_id", "rental_id", "amount", "payment_date"]}
{"language": ["language_id", "name", "last_update"]}
{"staff": ["staff_id", "first_name", "last_name", "address_id", "email", "store_id", "active", "username", "password", "last_update", "picture"]}
{"category": ["category_id", "name", "last_update"]}
{"customer": ["customer_id", "store_id", "first_name", "last_name", "email

## Probando la implementación en un solo paso

In [9]:
tabla_resultado = llm_to_pagila.nlp_to_sql(consulta_nlp=consulta_pagila_03, max_tokens= 1000)
simular_respuesta_generativa(tabla_resultado)

El archivo ya existe en: ../data/nlp_to_sql/pagila_simple_metadata.pickle
El archivo ya existe en: ../data/nlp_to_sql/pagila_full_metadata.pickle

Proceso de almacenamiento de metadatos para pagila completado.


El codigo sql que se ejecutará para responder tu consulta:
WITH ActorCount AS (
    SELECT
        fa.actor_id,
        COUNT(fa.film_id) AS film_count
    FROM
        film_actor fa
    GROUP BY
        fa.actor_id
    ORDER BY
        film_count DESC
    LIMIT 1
),
TopActorFilms AS (
    SELECT
        fa.film_id,
        a.actor_id,
        a.first_name,
        a.last_name
    FROM
        film_actor fa
    JOIN ActorCount ac ON
        fa.actor_id = ac.actor_id
    JOIN actor a ON
        fa.actor_id = a.actor_id
)
SELECT
    f.title,
    f.rental_rate,
    taf.first_name,
    taf.last_name
FROM
    TopActorFilms taf
JOIN film f ON
    taf.film_id = f.film_id;


Aqui tienes el resultado de tu consulta:
|    | title                   |   rental_rate | first_name   | last_na

## Le volvemos a pasar la tabla en formato texto al llm para que nos comente los resultado

In [10]:
informe = LLMsToDataBase.informe_resultado(consulta_usuario= consulta_pagila_03, tabla_texto=tabla_resultado, max_tokens_respuesta=2000 )
simular_respuesta_generativa(informe)

El actor con más películas en la tabla es Gina Degeneres. Las películas en las que ha trabajado y sus precios de alquiler son:

- Películas con un precio de alquiler de 0.99:
  - Excitement Eve
  - Gandhi Kwai
  - Gunfight Moon
  - Hook Chariots
  - Impact Aladdin
  - Indian Love
  - Intrigue Worst
  - Louisiana Harry
  - Midnight Westward
  - Mummy Creatures
  - Sierra Divide
  - Spirited Casualties
  - Storm Happiness

- Películas con un precio de alquiler de 2.99:
  - Bed Highball
  - Chaplin License
  - Chariots Conspiracy
  - Clueless Bucket
  - Darkness War
  - Deer Virginian
  - Hyde Doctor
  - License Weekend
  - Magnificent Chitty
  - Metal Armageddon
  - Searchers Wait
  - Window Side

- Películas con un precio de alquiler de 4.99:
  - Calendar Gunfight
  - Chamber Italian
  - Coldblooded Darling
  - Coneheads Smoochy
  - Dogma Family
  - Elephant Trojan
  - Frisco Forrest
  - Goodfellas Salute
  - Hall Cassidy
  - Heartbreakers Bright
  - Movie Shakespeare
  - Open African
 

In [ ]:
llm_to_pagila.close_connection()

In [11]:
from calcular_tokens import num_tokens_from_string

print(num_tokens_from_string(informe))
print(num_tokens_from_string(tabla_resultado))

378
1010
